In [260]:
! pip install bs4
! pip install fuzzywuzzy
! pip install geohash
! pip install python-geohash
! pip install pygeohash
! pip install python-Levenshtein
! pip install proximityhash

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/9c/e2/1a3507af7c8f91f8a4975d651d4aeb6a846dfdf74713954186ade4205850/python-geohash-0.8.5.tar.gz
  Running setup.py bdist_wheel for python-geohash: started
  Running setup.py bdist_wheel for python-geohash: finished with status 'error'
  Complete output from command C:\Users\Acer\Anaconda3\envs\rstudio\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-tcdcgjc8\\python-geohash\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Acer\AppData\Local\Temp\pip-wheel-cpvkxwkh --python-tag cp37:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  copying geohash.py -> build\lib.win-amd64-3.7
  copying quadtree.py -> build\lib.win-amd64-3.7
  copying jpgrid.py -> build\lib.win-amd64-3.7
  copying jpiarea.py -> build\li

  Failed building wheel for python-geohash
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
Command "C:\Users\Acer\Anaconda3\envs\rstudio\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-tcdcgjc8\\python-geohash\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Acer\AppData\Local\Temp\pip-record-8cp8xg8_\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Acer\AppData\Local\Temp\pip-install-tcdcgjc8\python-geohash\
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/42/a9/d1785c85ebf9b7dfacd08938dd028209c34a0ea3b1bcdb895208bd40a67d/python-Levenshtein-0.12.0.tar.gz
  Running setup.py bdist_wheel for python-Levenshtein: started
  Running setup.py bdist_wheel for python-Levenshtein: finished with status 'error'
  Complete output from command C:\Users\Acer\Anaconda3\envs\rstudio\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-78_7e2qn\\python-Levenshtein\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Acer\AppData\Local\Temp\pip-wheel-5m86t60n --python-tag cp37:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\Levenshtein
  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.7\Levenshtein
  copying Levenshtein\__init__.py 

  Failed building wheel for python-Levenshtein
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
Command "C:\Users\Acer\Anaconda3\envs\rstudio\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-78_7e2qn\\python-Levenshtein\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Acer\AppData\Local\Temp\pip-record-1931k9kx\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Acer\AppData\Local\Temp\pip-install-78_7e2qn\python-Levenshtein\
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import urllib.request
import requests
import numpy as np
from urllib.request import Request, urlopen
import pandas as pd
import re
from bs4 import BeautifulSoup
from random import randint
import time
import math
import csv
import string
# import json
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select 
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import Geohash
# import pygeohash as pgh
# import proximityhash

C:\Users\Acer\Anaconda3\envs\rstudio\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [67]:
#Parsing webpage
url_front = "https://giantonline.com.sg/"
req = Request(url_front, headers={'User-Agent': 'Chrome//5.0'})
page = urlopen(req)
soup = BeautifulSoup(page)
all_left_menu_item = soup.find('div', {'class' : 'left-menu'})


#Converting cateogries into string for regex & to generate the broad categories
broad_categories = all_left_menu_item.findAll('a', {'class' : 'department'})
for i in range(0,len(broad_categories)):
    broad_categories[i] = str(broad_categories[i])
    broad_categories[i] = re.findall(r'">(.*?)</a>', broad_categories[i])[0]
    broad_categories[i] = broad_categories[i].replace(", ", "-")
    broad_categories[i] = broad_categories[i].replace(" &amp; ", "-")
    if broad_categories[i][-1] == " ":
        broad_categories[i] = broad_categories[i][0:-1]
    broad_categories[i] = broad_categories[i].replace(" ", "-")

#Cos giant fked-up, name their product differently
broad_categories[1] = 'Beers-Wines-Spirits'

#Converting broad categories into smaller categories. For e.g. 'Rice' is under 'Food Pantry'    
narrow_categories = all_left_menu_item.findAll('li', {'class' : 'dropdown category'})
narrow_categories = narrow_categories[1:]
for i in range(0,len(narrow_categories)):
    narrow_categories[i] = str(narrow_categories[i])
    narrow_categories[i] = re.findall(r'">(.*?)</a>', narrow_categories[i])[1:]
    for x in range(0,len(narrow_categories[i])):
        narrow_categories[i][x] = narrow_categories[i][x].replace(", ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(" &amp; ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(" ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(".", "")
        narrow_categories[i][x] = narrow_categories[i][x].replace("'", "")

#Again cos giant fked-up, name their products differently
narrow_categories[0][-1] = 'international-groceries'
narrow_categories[2][-1] = 'chocolate-malts-adult-milk'

# Combining the list of URLs to form the webpage for only the first page        
combine_categories = list()
for i in range(0,len(broad_categories)):
    for x in range(0, len(narrow_categories[i])):
        word = str('https://giantonline.com.sg/' + str(broad_categories[i]) + '/' + str(narrow_categories[i][x]) + '/')
        combine_categories.append(word)

# Coming up with the full list of product with page
combine_cat_page = list()
for x in range(0,len(combine_categories)):
    for i in range(0,30):
        webpagelink = combine_categories[x] + '?Product_page=' + str(i)
        combine_cat_page.append(webpagelink)



print(len(combine_cat_page))
# print(all_left_menu_item)
# len(narrow_categories)
# print(narrow_categories[1])
# broad_categories
print(combine_cat_page)

2640
['https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=0', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=1', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=2', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=3', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=4', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=5', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=6', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=7', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=8', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=9', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=10', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=11', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=12', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=13', 'https://giantonline.com.sg/Food-Pantry/Rice/?Product_page=14', 'https://giantonline.com.sg/Food-Pantry/Rice

In [289]:
#Conducting actual webscraping
product_details = {}
df = pd.DataFrame()
for i in range(0,len(combine_cat_page)):
    req = Request(combine_cat_page[i], headers={'User-Agent': 'Chrome//5.0'})
    page = urlopen(req)
    soup = BeautifulSoup(page)
    items = soup.findAll('li', {'class' : 'col-xs-6 col-sm-4 col-md-3 col-lg-2 open-product-detail algolia-click'})
    current_page = items[0].find('a', {'class' : "product-link"}).get_text()
    if i == 0 or current_page != previous_page:
        for x in range(0,len(items)):
            try:
                #image source
                src = str(items[x].find('img'))
                product_details['Image'] = re.findall(r'src="(.*?)"', src)[0]
            except:
                src = "No Image Link"
                product_details['Image'] = src
            try:
                #brand name
                brand = items[x].find('a', {'class' : 'to-brand-page'}).get_text()
                product_details['Brand'] = brand
            except:
                brand = 'Brandless'
                product_details['Brand'] = brand
            try:
                #further product info
                details = items[x].find('a', {'class' : "product-link"}).get_text()
                product_details['Further Details'] = details
            except:
                details = 'No Further Details'
                product_details['Further Details'] = details
            try:
                #weight of product
                size = items[x].find('span', {'class' : 'size'}).get_text()
                product_details['Size'] = size
            except:
                size = 'No Size'
                product_details['Size'] = size
            try:
                #any discounts
                discount = items[x].find('span', {'class' : 'product-discount-label'}).get_text()
                product_details['Discount'] = discount
            except:
                discount = 'No Discount'
                product_details['Discount'] = discount
            try:
                #Happens when there's a discount
                items_x = str(items[x])
                new_price = re.findall(r'<div class="price product-price red" data-price="(.*?)">', items_x)[0]
                old_price = re.findall(r'<span class="price old-price">(.*?)</span>', items_x)[0]
                product_details['Current Price'] = new_price
                product_details['Old Price'] = old_price
            except:
                try:
                    new_price = items[x].find('div', {'class' : 'price product-price'}).get_text()
                    product_details['Current Price'] = new_price
                    product_details['Old Price'] = 'No Discount'
                except:
                    new_price = items[x].find('div', {'class' : 'content_price'}).get_text().rstrip().lstrip()
                    product_details['Current Price'] = new_price
                    product_details['Old Price'] = 'No Discount'
            try:
                #link to goods under the brand
                brand_goods = str(items[x].find('a', {'class' : 'to-brand-page'}))
                brand_goods = re.findall(r'href="(.*?)">', brand_goods)[0]
                product_details['Brands Goods'] = brand_goods
            except:
                product_details['Brands Goods'] = 'No Page for Brand Goods'
            try:
                # Links to more details
                more_details = str(items[x])
                more_details = re.findall(r'<a class="product-link" href="(.*?)">', more_details)[0]
                more_details = 'https://giantonline.com.sg' + more_details
                product_details['Link to More Details'] = more_details
            except:
                product_details['Link to More Details'] = 'No Link'
            try:
                outstock = items[13].find('div', {'class' : 'btn add-cart btn-default out-of-stock'}).get_text().rstrip().lstrip()
                product_details['Stock'] = outstock
            except:
                product_details['Stock'] = "In Stock"
            try:
                comb_info = combine_cat_page[0].split('_page=')
                page_number = comb_info[0]
                comb_cat = re.findall(r'https://giantonline.com.sg/(.*?)/?Product', comb_info[0])
                broad_cat = comb_cat[0].split('/')[0]
                narrow_cat = comb_cat[0].split('/')[1]
                product_details['Page Number'] = page_number
                product_details['Broad Category'] = broad_cat
                product_details['Narrow Category'] = narrow_cat
            except:
                product_details['Page Number'] = 'No Page'
                product_details['Broad Category'] = 'No Broad Cat'
                product_details['Narrow Category'] = 'No Narrow Cat'
            df = df.append(product_details, ignore_index = True)
        previous_page = current_page     
    print(i)



1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909


In [ ]:
print(df)
df.to_csv('Giant Supermarket Scrape.csv', sep=',')